# Research Request - GTFS Digest: Add Rail and Ferry Operators. #1386
# Focus on Rail here. 

* Why is Amtrak "schedule_only"? I thought it had a real time component?
    * 3/3: no reports.calitp.org indicates it has no real time data. 
* Understand why Southern California Regional Rail Authority 	 & San Bernardino County Transportation Authority 	are still "vp_only" after all the other operators have been fixed.
* Visualize rail routes differently for operators with both? 

In [1]:
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
import geopandas as gpd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS, PROJECT_CRS
from shared_utils import catalog_utils, portfolio_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = [rt_dates.DATES["feb2025"]]

In [4]:
analysis_date = rt_dates.DATES["feb2025"]

In [5]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [6]:
EXPORT = GTFS_DATA_DICT.schedule_tables.route_typologies

In [7]:
route_typologies = pd.read_parquet(f"{SCHED_GCS}{EXPORT}_{analysis_date}.parquet")

In [34]:
schd_vp_df = pd.read_parquet(
    schd_vp_url,
)

In [35]:
schd_vp_df.columns

Index(['schedule_gtfs_dataset_key', 'direction_id', 'time_period',
       'avg_scheduled_service_minutes', 'avg_stop_miles', 'n_scheduled_trips',
       'frequency', 'is_express', 'is_rapid', 'is_rail', 'is_coverage',
       'is_downtown_local', 'is_local', 'service_date', 'typology',
       'minutes_atleast1_vp', 'minutes_atleast2_vp',
       'total_rt_service_minutes', 'total_scheduled_service_minutes',
       'total_vp', 'vp_in_shape', 'is_early', 'is_ontime', 'is_late',
       'n_vp_trips', 'vp_per_minute', 'pct_in_shape',
       'pct_rt_journey_atleast1_vp', 'pct_rt_journey_atleast2_vp',
       'pct_sched_journey_atleast1_vp', 'pct_sched_journey_atleast2_vp',
       'rt_sched_journey_ratio', 'avg_rt_service_minutes', 'sched_rt_category',
       'speed_mph', 'route_long_name', 'route_short_name',
       'route_combined_name', 'route_id', 'base64_url',
       'organization_source_record_id', 'organization_name',
       'caltrans_district', 'route_primary_direction', 'name',
       '

In [9]:
# Filter for Jan and Feb
schd_vp_df2 = schd_vp_df.loc[
    (schd_vp_df.service_date == "2025-02-12")
]

In [10]:
# Drop duplicates
schd_vp_df3 = (
    schd_vp_df2[
        [
            "schedule_gtfs_dataset_key",
            "organization_name",
            "service_date",
            "sched_rt_category",
            "caltrans_district",
        ]
    ]
    .drop_duplicates(subset=["organization_name"])
    .sort_values(by=["organization_name"])
)

In [11]:
schd_vp_df3.sched_rt_category.value_counts()

schedule_and_vp    104
schedule_only       86
vp_only              2
Name: sched_rt_category, dtype: int64

## Trying to undestand why Southern California Regional Rail Authority 	 & San Bernardino County Transportation Authority 	are still "vp_only" after all the other operators have been fixed.

In [12]:
vp_only_ops = list(
    schd_vp_df3.loc[
        schd_vp_df3.sched_rt_category == "vp_only"
    ].organization_name.unique()
)

In [13]:
vp_only_ops_df = schd_vp_df.loc[schd_vp_df.organization_name.isin(vp_only_ops)]

In [14]:
vp_only_ops_sched_keys = list(vp_only_ops_df.schedule_gtfs_dataset_key.unique())

In [15]:
vp_only_ops_df.groupby(["caltrans_district","organization_name", "sched_rt_category"]).agg(
    {"service_date": "max"}
)

service_date
caltrans_district          organization_name                              sched_rt_category             
07 - Los Angeles           San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only       2024-06-12
                                                                          vp_only             2024-12-11
                                                                          schedule_and_vp     2024-06-12
07 - Los Angeles / Ventura San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp            NaT
08 - San Bernardino        San Bernardino County Transportation Authority schedule_only       2024-06-12
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp     2024-06-12
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT

## Go back to  `schedule_stats_by_route_direction`

In [16]:
import sys

sys.path.append("../gtfs_funnel/")
import schedule_stats_by_route_direction

In [17]:
route_group_merge_cols = [
            "schedule_gtfs_dataset_key", 
            "route_id", 
            "direction_id"
        ]
        

### The two operators are already disappearing in the `trip_metrics` line 203. 

In [18]:
trip_metrics = schedule_stats_by_route_direction.assemble_scheduled_trip_metrics(
    analysis_date, GTFS_DATA_DICT
)

In [19]:
trip_metrics = trip_metrics.loc[
    trip_metrics.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [25]:
len(trip_metrics)

0

### Operators are already gone in `STOP_TIMES_FILE` in line 21.

In [21]:
STOP_TIMES_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [77]:
STOP_TIMES_FILE

'stop_times_direction'

In [78]:
GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

'stop_times_direction'

In [79]:
df = gpd.read_parquet(
        f"{RT_SCHED_GCS}{STOP_TIMES_FILE}_{analysis_date}.parquet"
    )

In [82]:
analysis_date

'2025-02-12'

In [80]:
len(df)

4326927

In [83]:
vp_only_ops_sched_keys

['759ad28de7d4bb8b2bf9bb7d83655100', 'c4092405159366c705b62df938293a4e']

In [84]:
df2.columns

Index(['feed_key', 'stop_id', 'stop_sequence', 'schedule_gtfs_dataset_key',
       'trip_instance_key', 'shape_array_key', 'stop_name', 'geometry',
       'stop_meters', 'prior_stop_sequence', 'subseq_stop_sequence',
       'stop_primary_direction', 'stop_pair', 'stop_pair_name'],
      dtype='object')

In [81]:
df2 = df.loc[
    df.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [24]:
len(df)

0

### Go back to `gtfs_funnel/stop_times_with_direction.py`

#### Per Tiffany's advice, check `trips`. 

In [39]:
trip_scheduled_col = [
        "route_id",
        "trip_instance_key",
        "gtfs_dataset_key",
        "shape_array_key",
        "direction_id",
        "route_long_name",
        "route_short_name",
        "route_desc",
        "name",
        "feed_key"
    ]

In [40]:
trips_df = helpers.import_scheduled_trips(
        analysis_date, 
        columns = trip_scheduled_col,
        get_pandas = True
    )

In [41]:
trips_df = trips_df.loc[
    trips_df.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [31]:
len(trips_df)

202

In [42]:
trips_df.head(1)

,route_id,trip_instance_key,schedule_gtfs_dataset_key,shape_array_key,direction_id,route_long_name,route_short_name,route_desc,name,feed_key
114833,San Bernardino Line,e0e22a031655b8c4eb6eb418166b567c,c4092405159366c705b62df938293a4e,None,0.00,Metrolink San Bernardino Line,None,None,Metrolink Schedule,8fc9cfe86b4e9e8c7cf508b2486605f1


In [43]:
feed_keys_list = list(trips_df.feed_key.unique())

In [58]:
feed_keys_list

['8fc9cfe86b4e9e8c7cf508b2486605f1']

In [59]:
trips = helpers.import_scheduled_trips(
        analysis_date,
        columns = ["gtfs_dataset_key", "feed_key", 
                   "trip_id", "trip_instance_key", 
                   "shape_array_key"
                  ],
        get_pandas = True
    )

In [60]:
trips = trips.loc[
    trips.feed_key.isin(feed_keys_list)
]

In [61]:
len(trips)

202

#### Stops

In [44]:
stops = helpers.import_scheduled_stops(
        analysis_date,
        columns = ["feed_key", "stop_id", "stop_name", "geometry"],
        crs = PROJECT_CRS,
        get_pandas = True
    )

In [46]:
stops = stops.loc[
    stops.feed_key.isin(feed_keys_list)
]

In [47]:
len(stops)

66

#### Stop_times - everything is here. 

In [50]:
stop_times = helpers.import_scheduled_stop_times(
        analysis_date,
        columns = ["feed_key", "trip_id", "stop_id", "stop_sequence"],
        get_pandas = True
    )

In [51]:
stop_times = stop_times.loc[
    stop_times.feed_key.isin(feed_keys_list)
]

In [52]:
len(stop_times)

2053

In [54]:
import stop_times_with_direction

In [55]:
stop_times2 = stop_times.pipe(stop_times_with_direction.keep_first_trip, analysis_date)

In [56]:
len(stop_times2)

2053

In [57]:
stop_times2.feed_key.unique()

array(['8fc9cfe86b4e9e8c7cf508b2486605f1'], dtype=object)

#### Line 47 merge

In [62]:
st_with_trip = pd.merge(
        stop_times,
        trips,
        on = ["feed_key", "trip_id"],
        how = "inner"
    )

In [63]:
len(st_with_trip)

2053

In [64]:
st_with_stop = pd.merge(
        st_with_trip,
        stops,
        on = ["feed_key", "stop_id"],
        how = "inner"
    ).drop(columns = ["trip_id"])

In [65]:
len(st_with_stop)

2053

In [66]:
st_with_stop = gpd.GeoDataFrame(
        st_with_stop, geometry = "geometry", crs = PROJECT_CRS
    )

### `stop_times_with_direction/prep_scheduled_stop_times` everything is in order here.

In [67]:
scheduled_stop_times = stop_times_with_direction.prep_scheduled_stop_times(analysis_date)

In [68]:
scheduled_stop_times = scheduled_stop_times.loc[
    scheduled_stop_times.feed_key.isin(feed_keys_list)
]

In [69]:
len(scheduled_stop_times)

2053

### Continue with Line 251 in `stop_times_with_direction`

In [70]:
trip_cols = ["trip_instance_key"]
trip_stop_cols = ["trip_instance_key", "stop_sequence"]

In [71]:
df = stop_times_with_direction.find_prior_subseq_stop_info(
        scheduled_stop_times,
        analysis_date,
        trip_cols = trip_cols,
        trip_stop_cols = trip_stop_cols
    ).sort_values(
        trip_stop_cols
    ).reset_index(drop=True)

In [72]:
len(df)

2053